# Model Component Example

This notebook outlines how to implement an example model component that can be used in estimating connectivity costs.
We'll work through functional examples of how you can think of components, but let's first define what a model component is.

**Cost Model Component**: an implementation of a class in the modeling library that performs a cost calculation with the following properties:
* It is independently configurable (e.g. has an explicit set of configurable cost driver parameter that dictate its output)
* Exposes a `run` method that runs the model, into which the data client is passed (the data client is of type `ModelDataSpace`). This means all the avaible data (schools, fiber data, and cellular data) are availble to the model when it runs
* Returns the results inside of an object called `CostResultSpace` which contains the costs attributed to all the schools that the data space has access to

Existing model components are implemented as standalone classes in `giga.models.components`.
In order to integrate these components into the application, you will need to perform two additional steps in addition to implementing the components:
1. Integrate the component into a runnable scenario (see `giga.models.scenarios.priority_scenario` for an example)
2. Create a UI component that can manage parameter inputs for this component (see `giga.viz.notebooks.parameters.groups.fiber_technology_parameter_manager.py`)

Once these are complete you can use the component in the application.

## Creating a Sample Model

Let's create a sample model component that will calculate a hypothetical cost using some simple rules.
As an example we'll implement a computation that exercises various cost estimation properties:
* Constant capex dictated by `cost_parameter_a`
* Variable opex dictated by `cost_parameter_b` for connected schools and `cost_parameter_c` for unconnected schools

We'll also add a configuration model that holds the parameters above.

In [ ]:
from pydantic import BaseModel, validate_arguments

from giga.utils.progress_bar import progress_bar as pb
from giga.schemas.output import CostResultSpace, SchoolConnectionCosts
from giga.utils.logging import LOGGER
from giga.data.space.model_data_space import ModelDataSpace

class SampleTechnologyCostConf(BaseModel):
    """
        Sample cost drivers all in USD
    """
    cost_parameter_a: float  # the cost of capex per school
    cost_parameter_b: float  # the cost of opex for a connected school
    cost_parameter_c: float  # the cost of opex for an unconnected school


class SampleCostModel:
    
    
    def __init__(self, config: SampleTechnologyCostConf):
        self.config = config

    def compute_cost(self, school: ModelDataSpace) -> SchoolConnectionCosts:
        # if connected assign cost_parameter_b to opex otherwise cost_parameter_x
        opex = self.config.cost_parameter_b if school.connected else self.config.cost_parameter_c
        return SchoolConnectionCosts(
            school_id=school.giga_id,
            capex=self.config.cost_parameter_a,
            capex_provider=self.config.cost_parameter_a,
            capex_consumer=0.0,
            opex=opex,
            opex_provider=0.0, # assume no cost attribution to provider
            opex_consumer=opex,
            technology="None",  # a valid technology type can be None
        )

    @validate_arguments(config=dict(arbitrary_types_allowed=True))
    def run(
        self, data_space: ModelDataSpace, progress_bar: bool = False
    ) -> CostResultSpace:
        """
        Computes a cost table for schools present in the data_space input
        """
        LOGGER.info(f"Starting Sample Cost Model")
        # we can access different types of data in the data_space client, let's grab the schools
        schools = data_space.school_entities
        iterable = pb(schools) if progress_bar else schools # creates a progress bar
        costs = [self.compute_cost(s) for s in iterable]
        return CostResultSpace(
            technology_results={"model_type": "Sample"}, cost_results=costs, tech_name="sample"
        )


## Running the Sample Model

Let's now run the sample model, we'll first load in the configuration, and create the data space we need to drive the model.
We'll then initialize the configuration for the model we've just defined and run it to generate our cost results.

In [ ]:
from giga.app.config_client import ConfigClient
from giga.app.config import get_country_default

# Let's load in the global app configuration
workspace = 'workspace' # the workspace where data for each country of interest can be found
country = 'RWA'

global_config = ConfigClient.from_country_defaults(
                    get_country_default(country, workspace=workspace)
                )

In [ ]:
config = SampleTechnologyCostConf(cost_parameter_a=1_000,
                                  cost_parameter_b=50,
                                  cost_parameter_c=100)

model = SampleCostModel(config)
data_space = ModelDataSpace(global_config.local_workspace_data_space_config)

costs = model.run(data_space, progress_bar=True)

In [ ]:
# let's look at one of the results
costs.cost_results[0].dict()

For an example of this implementation in the library see `giga.models.components.sample_cost_model.py`.

## Implementing More Complex Models

If you would like to implement a more complex that leverages multiple data sources you can use the data_space client and the information it has available to it:

# ModelDataSpace

Client for providing the necessary external data needed to drive the cost models. Includes:

- Schools
- Fiber Nodes
- Cell Towers

## Properties

### all_schools

Accessor for school entities - includes coordinates, school ids, and other metadata such as electricity availability, connectivity quality, etc.

### schools

Accessor for school entities - includes coordinates, school ids, and other metadata such as electricity availability, connectivity quality, etc.

### schools_with_fiber_coordinates

Accessor for school coordinates only those that are connected with fiber.

### school_coordinates

Accessor for unconnected school coordinates - id, lat, lon information.

### schools_with_electricity_coordinates

Accessor for school coordinates only those that are connected to the electric grid.

### school_entities

Accessor for unconnected school entities - includes coordinates, school ids, and other metadata such as electricity availability, connectivity quality, etc.

### all_school_entities

Accessor for school entities - includes coordinates, school ids, and other metadata such as electricity availability, connectivity quality, etc.

### fiber_map

Accessor for the fiber map, which is a coordinate table containing the coordinates of all fiber nodes in the region of interest.

### fiber_coordinates

Accessor to fiber coordinates - a list of id, lat, lon.

### cell_tower_map

Accessor for the cell tower map - a coordinate table containing the coordinates and metadata of all cell towers in the region of interest.

### cell_tower_coordinates

Accessor to cell tower coordinates - a list of id, lat, lon.

### fiber_cache

Accessor for the fiber distance cache - a table of distances between all schools and fiber nodes. This includes pairwise nearest distances between school/school pairs as well.

### cellular_cache

Accessor for the cellular distance cache - a table of distances between all schools and cell towers.

### p2p_cache

Accessor for the p2p distance cache - a table of distances between all schools and cell towers. Includes line of sight information.